In [2]:
# ===========================
# Vibe Matcher Prototype
# ===========================

!pip install --quiet pandas scikit-learn
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Markdown, display
import time

# ===========================
# 2. Introduction (Nexora)
# ===========================
display(Markdown("""
# Vibe Matcher Prototype

**Introduction:**
At Nexora, AI-driven recommendation systems help users discover products that match their personal vibe and style.
This notebook demonstrates a mini prototype that matches user queries to fashion products using embeddings and similarity scores.
"""))


# 3. Data Preparation

data = [
    {"name": "Boho Dress", "desc": "Flowy, earthy tones for festival vibes", "vibes": ["boho", "cozy"]},
    {"name": "Urban Hoodie", "desc": "Bold streetwear style with energetic colors", "vibes": ["urban", "energetic"]},
    {"name": "Elegant Blazer", "desc": "Tailored sophistication for office chic", "vibes": ["chic", "formal"]},
    {"name": "Casual Sneakers", "desc": "Comfortable shoes for everyday urban adventures", "vibes": ["casual", "urban"]},
    {"name": "Cozy Sweater", "desc": "Soft knit, warm vibes for winter lounging", "vibes": ["cozy", "relaxed"]},
]
df = pd.DataFrame(data)
display(Markdown("## Product DataFrame"))
display(df)

# ===========================
# 4. Embeddings (Mock)
# ===========================
display(Markdown("""
**Note:**
Embeddings are mocked due to OpenAI quota limits.
With an active API key, you can replace `get_embedding` with OpenAI API to generate real embeddings:

```python
from openai import OpenAI
client = OpenAI(api_key="YOUR_API_KEY")
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding
```
"""))

# Mock embeddings for products
np.random.seed(42)
embedding_dim = 50
df['embedding'] = [np.random.rand(embedding_dim) for _ in range(len(df))]

# Mock embedding function for queries
def get_embedding(text):
    vec = np.array([(sum(ord(c) for c in text)+i) % 100 / 100 for i in range(embedding_dim)])
    return vec

# ===========================
# 5. Matching function
# ===========================
def match_vibe(query, top_k=3):
    query_emb = get_embedding(query)
    sim_scores = [cosine_similarity([query_emb], [e])[0][0] for e in df['embedding']]
    df['sim_score'] = sim_scores
    top_matches = df.sort_values(by='sim_score', ascending=False).head(top_k)

    if top_matches['sim_score'].max() < 0.5:
        display(Markdown("**No strong match found. Showing popular items instead:**"))
        return df.head(top_k)[['name', 'desc', 'sim_score']]

    return top_matches[['name', 'desc', 'sim_score']]

# ===========================
# 6. Testing & Evaluation
# ===========================
queries = ["energetic urban chic", "cozy winter vibes", "festival boho look"]
for q in queries:
    display(Markdown(f"### Query: '{q}'"))
    start_time = time.time()
    top = match_vibe(q)
    display(top)
    print(f"Latency: {time.time() - start_time:.4f} seconds")

# ===========================
# 7. Reflection / Improvements
# ===========================
reflection = [
    "Integrate OpenAI embeddings later for real results.",
    "Use Pinecone or FAISS for scalable, fast vector search.",
    "Fine-tune embeddings for fashion-specific vocabulary.",
    "Experiment with multi-modal input (text + image).",
    "Track user feedback to improve vibe tagging and accuracy."
]

display(Markdown("## Reflection / Improvements"))
for r in reflection:
    print(f"- {r}")


# Vibe Matcher Prototype

**Introduction:**
At Nexora, AI-driven recommendation systems help users discover products that match their personal vibe and style.
This notebook demonstrates a mini prototype that matches user queries to fashion products using embeddings and similarity scores.


## Product DataFrame

,name,desc,vibes
0,Boho Dress,"Flowy, earthy tones for festival vibes","[boho, cozy]"
1,Urban Hoodie,Bold streetwear style with energetic colors,"[urban, energetic]"
2,Elegant Blazer,Tailored sophistication for office chic,"[chic, formal]"
3,Casual Sneakers,Comfortable shoes for everyday urban adventures,"[casual, urban]"
4,Cozy Sweater,"Soft knit, warm vibes for winter lounging","[cozy, relaxed]"



**Note:**
Embeddings are mocked due to OpenAI quota limits.
With an active API key, you can replace `get_embedding` with OpenAI API to generate real embeddings:

```python
from openai import OpenAI
client = OpenAI(api_key="YOUR_API_KEY")
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding
```


### Query: 'energetic urban chic'

...

### Query: 'festival boho look'

,name,desc,sim_score
0,Boho Dress,"Flowy, earthy tones for festival vibes",0.703768
3,Casual Sneakers,Comfortable shoes for everyday urban adventures,0.690611
1,Urban Hoodie,Bold streetwear style with energetic colors,0.663928


Latency: 0.0192 seconds


## Reflection / Improvements

- Integrate OpenAI embeddings later for real results.
- Use Pinecone or FAISS for scalable, fast vector search.
- Fine-tune embeddings for fashion-specific vocabulary.
- Experiment with multi-modal input (text + image).
- Track user feedback to improve vibe tagging and accuracy.
